In [ ]:
import os
import re

import pandas as pd
import numpy as np
#import warnings
#warnings.filterwarnings("ignore")


Provide path to directory where the files are stored. This depends highly on how things are set up on your system. I downloaded the files from Qualtrics and put them in a dedicated directory. They are named e.g. baseline_1_text.csv, baseline_2_text.csv etc, same for the follow-up files. The way this program is set up is that it will ask you for input. When the prompt appears, just enter the name of the file you want to work on. Many ways to skin a cat though - this is just one way of dealing with the files.

In [ ]:
home = 1
if home:
    infiledir = r"C:\Users\Possible_online_studies\NLP_expressive_writing\analysis"
else:
    infiledir = r"P:\EW_analysis\analysis"
filename = input("Please enter input filename, including file extension.")

These two files are needed to get the ADNM scores for the baseline questionnaires. More on this will follow below.

In [ ]:
screening_df = pd.read_csv(os.path.join(infiledir, 'screening.csv'),skiprows = [0,2]) # read in screening and randomisation files
randomisation_df = pd.read_csv(os.path.join(infiledir, 'randomisation.csv'))

Here, you enter the assessment number you are working on (so 1,2 3 or 4).

In [ ]:
# assessment number
assessment_number = input("Please enter the assessment number (1, 2, 3 or 4")

Here, you specify where you want the output to go and what you want it to be called.

In [ ]:
# Save output here
if home:
    output_dir = r"C:\Users\Luzia T\UCL\WorkingFromHome\Possible_online_studies\NLP_expressive_writing\analysis\Processed_2"
else:
    output_dir = r"P:\EW_analysis\analysis\Processed_2"
outfilename = input("Please enter desired output file name, including file extension.")

Now we read in the Qualtrics file. This is pretty basic, I'm not really using any of the filter options other than the skiprows argument. This is to skip rows 0 and 2, which are just confusing and don't hold any additional information that I want.

In [ ]:
test_df= pd.read_csv(os.path.join(infiledir, filename),skiprows = [0,2])

One of the trickier things of dealing with this data set is that there are duplicate records for certain ids. There may be many reasons for this. For instance, a participant may have started an assessment, encountered issues, and stopped, then started again, resulting in multiple records for the same id. Some participants may not have been able to click through to the Gorilla task, resulting in 'complete' status in Qualtrics, but no Gorilla record. Although most of these people got a link to the assessment only, some participants just recompleted the entire thing, resulting in multiple records for the same person. Here's how we will proceed in removing duplicate ids:
(1) Remove any rows with incomplete status
(2) Remove rows for duplicate ids if they contain <70% of valid data
(3) Check Qualtrics ID against LDI record. If one of them exists but the other does not, use the record with the existing MST record.
(4) If all duplicates for a single ID contain data, use the one with the later completion date/time.

In [ ]:
# Find incomplete records and duplicates
test_df.rename(columns = {'Please enter the participant number you were sent.': 'id'},inplace = True)
incomplete = test_df[test_df['Finished']==False]['id']
print('Did not finish:', incomplete)

Did not finish: 11      NaN
41     68.0
87    112.0
Name: id, dtype: float64


Drop rows with 'incomplete' status.

In [ ]:
test_df.drop(labels = incomplete.index, axis = 0, inplace = True)

Now check for duplicates.

In [ ]:
duplicates = test_df[test_df.duplicated(subset = 'id')]
print(duplicates['id'])

67    107.0
80    219.0
Name: id, dtype: float64


Now we check for rows with duplicate ids and >70% of invalid (nan or 0) data. We do this for each duplicated id in turn, since we don't know how many times someone attempted the questionnaire.

In [ ]:
drop_labels = []
for count, pnum in enumerate(duplicates.id):
    dup_df = test_df[test_df.id==pnum]
    prop_nil = dup_df.replace(0,np.nan).isna().sum(axis = 1)/dup_df.shape[1]
    drop_ind = prop_nil[prop_nil>0.7]
    drop_labels.append(drop_ind)

if np.array(drop_labels).size==0:
    print('No empty rows to drop.')
else:
    test_df = test_df.drop(labels = drop_ind.index, axis = 0)
    print('dropped labels: ', np.array(drop_labels))
    

No empty rows to drop.


We now check for any remaining duplicated records. We then use the info from the LDI file to check if, for a given ID, one record has an existing MST result while the other(s) do/does not.

In [ ]:
duplicates = test_df[test_df.duplicated(subset = 'id',keep = False)]
print(duplicates['id'])

64    107.0
67    107.0
79    219.0
80    219.0
Name: id, dtype: float64


Load the LDI file from appropriate directory.

In [ ]:
if home:
    MST_dir = r"C:\Users\Luzia T\UCL\WorkingFromHome\Possible_online_studies\NLP_expressive_writing\analysis\Processed_data\MST"
else:
    MST_dir = r"P:\EW_analysis\analysis\Processed_data\MST"
ldi_name = input("Please enter the name of the file with the LDI results, including file extension. ")
LDI_df = pd.read_csv(os.path.join(MST_dir, ldi_name))
LDI_df.head()

,qualtrics_id,rec_part,ldi_part,targets,lures,rec_prob,ldi_prob,date_time,task_type
0,R_3E0TOIqjlwa684F,42,0,128,128,0.328125,0.000000,2020-10-28 16:29:00,1
1,R_2ZPsf7B81JXlKoE,60,3,128,128,0.468750,0.023438,2020-10-30 09:27:00,1
2,R_2qjSlzamK6c9Y66,54,9,128,128,0.421875,0.070312,2020-11-02 10:22:00,1
3,R_3q7FKiW4r0tYTu4,56,19,128,128,0.437500,0.148438,2020-11-02 11:55:00,1
4,R_O6vCXXxaVVrmeJP,61,23,128,128,0.476562,0.179688,2020-11-02 12:47:00,1


In [ ]:
# Get the qualtrics ids for the subjects with duplicate ids:
qualtrics_ids = test_df.loc[duplicates.index,'Response ID']
print(qualtrics_ids)
ids_exist = qualtrics_ids[qualtrics_ids.isin(LDI_df.qualtrics_id)]
print(ids_exist)
# now drop the records with existing qualtrics ids from the duplicates
duplicates.drop(labels = ids_exist.index, axis = 0, inplace = True)
# Finally, use the remaining indices from duplicates to drop these records from the dataframe:
test_df.drop(labels = duplicates.index, axis = 0, inplace = True)

64    R_3r36lzHPejmI1tK
67    R_2SDeL4CXliMT0Cu
79    R_cIsj3jgNgnx6iM9
80    R_28GQmOZwVVmGAES
Name: Response ID, dtype: object
67    R_2SDeL4CXliMT0Cu
80    R_28GQmOZwVVmGAES
Name: Response ID, dtype: object


Ok, now on to the last option. If we can't solve the duplicated id issue with any of the above, we will just use the record with the max date/time stamp as recorded by Qualtrics. 

In [ ]:
if test_df[test_df.duplicated(subset = 'id')].empty:
    print('No more duplicates in ids.')
else:
    print('More duplicates left.')
    duplicates = test_df[test_df.duplicated(subset = 'id', keep = False)]
    test_df.loc[:,'Start Date'] = pd.to_datetime(test_df.loc[:,'Start Date']) # convert to date time format
    for i, pnum in enumerate(duplicates.id.unique()):
        id_df = test_df.loc[test_df.id == pnum,['Start Date','id']]
        max_ind = id_df.where(id_df['Start Date']== id_df['Start Date'].max(),np.nan).dropna(how = 'all',axis = 0)
        duplicates.drop(labels = max_ind.index,axis = 0, inplace = True)
        print('Dropping ids: ',duplicates['id'])
    test_df.drop(labels = duplicates.index, axis = 0, inplace =  True)

No more duplicates in ids.


Next, we'll drop rows where the id field is empty. We can't do anything with data that don't have a participant number associated with them, so we'll chuck them out.

In [ ]:
test_df = test_df.drop(labels = test_df[test_df.id.isna()].index, axis = 0)
print(f'dropped {test_df[test_df.id.isna()].shape[0]} subjects due to missing id')

dropped 0 subjects due to missing id


We're about to get the info that we are going to use for the rest of the analysis from the df. The below file just holds the names of the start and end of all the questionnaires we want to include for further processing.
Bear in mind that again, this is one way of doing things. You could argue that, since we are dropping very few columns, we could just specify which cols we want to drop and then proceed to drop them. However, this way, we already have the column names for each of the questionnaires for the scoring bit later on.

In [ ]:
col_key = pd.read_csv(os.path.join(infiledir, 'survey_cols.csv'),index_col = 0)#pd.read_csv(os.path.join(infiledir,qcode_name))

In [ ]:
DASS_cols = test_df.loc[:,col_key.DASS['start']:col_key.DASS['end']].columns
ERQ_cols = test_df.loc[:,col_key.ERQ['start']:col_key.ERQ['end']].columns
RRS_cols = test_df.loc[:,col_key.RRS['start']:col_key.RRS['end']].columns
SOC_cols = test_df.loc[:,col_key.SOC['start']:col_key.SOC['end']].columns
GPAQ_cols = test_df.loc[:,col_key.GPAQ['start']:col_key.GPAQ['end']].columns
PTGI_cols = test_df.loc[:,col_key.PTGI['start']:col_key.PTGI['end']].columns
if 'follow' in filename:
    Demographics_cols = test_df.loc[:,col_key.Demographics_follow['start']:col_key.Demographics_follow['end']].columns
    ADNM_cols = test_df.loc[:,col_key.ADNM['start']:col_key.ADNM['end']].iloc[:,1:-1].columns
    # There's a typo in the follow-up version of this, so need to specify separately.
    PSQI_start = 'During the past month, how many hours of actual sleep did you get at night? (This may be different than the number of hours you spent in bed.) Please enter the average number of hours of sleep per night.'
    PSQI_cols = test_df.loc[:,PSQI_start:col_key.PSQI['end']].columns
    cols_to_keep = np.concatenate([Demographics_cols.values,ADNM_cols.values,DASS_cols.values,ERQ_cols.values,RRS_cols.values,SOC_cols.values,GPAQ_cols.values,PSQI_cols.values,PTGI_cols.values,['id','Response ID','Start Date']],axis = None)
else:
    Covid_cols = test_df.loc[:,col_key.Covid['start']:col_key.Covid['end']].columns
    Demographics_cols = test_df.loc[:,col_key.Demographics['start']:col_key.Demographics['end']].columns
    PSQI_cols = test_df.loc[:,col_key.PSQI['start']:col_key.PSQI['end']].columns
    cols_to_keep = np.concatenate([Demographics_cols.values,Covid_cols.values,DASS_cols.values,ERQ_cols.values,RRS_cols.values,SOC_cols.values,GPAQ_cols.values,PSQI_cols.values,PTGI_cols.values,['id','Response ID','Start Date']],axis = None)

In [ ]:
test_df = test_df.filter(items = cols_to_keep, axis = 1)
print(test_df.shape)

(86, 199)


There is also one additional thing we need to do before looking up the qualtrics ids. Some subjects had issues with the tasks. To allow them to complete just this bit without having to recomplete the entire thing, I made a separate qualtrics survey that just asked for their participant number and sent them straight through to Gorilla to complete the task. We'll now load the Qualtrics data from this and we'll replace the Qualtrics ID for those participants where this happened. Note that we are reading in the participant id column as float64. This is because we want it to be the same data type as the id column in test_df.

In [ ]:
MST_replaced_name = input('Please enter the replacement ID MST filename, including file extension: ')
MST_replaced = pd.read_csv(os.path.join(MST_dir,MST_replaced_name),dtype = {'Please enter your participant ID': np.float64})

Now we need to replace the Qualtrics IDs in our dataframe - if necessary. First of all, for easier manipulation, we'll set the index of the test_df dataframe to 'id'. Next, we will (i) check whether any ids need replacing and (ii) if yes, replace them. Here we are using a dict and then apply to accomplish this. You could also simply set the index to be id in the MST_replaced df.

In [ ]:
test_df.set_index('id',inplace = True)

In [ ]:
for count,part in enumerate(MST_replaced['Please enter your participant ID']):
    if part in test_df.index:
        #print(part)
        print('Qualtrics ID before replacement:', test_df['Response ID'][part])
        test_df['Response ID'][part] = MST_replaced['Response ID'][count]
        print('Qualtrics ID after replacement:', test_df['Response ID'][part])
    else:
        print('No need to replace id ', part)

Qualtrics ID before replacement: R_3HnvbjkUffyX6ff
Qualtrics ID after replacement: R_1jdK6kFzPV4b7fa
Qualtrics ID before replacement: R_22m37yYKGWwJOC4
Qualtrics ID after replacement: R_9WGNk5J81UbWsRb
No need to replace id  39.0
Qualtrics ID before replacement: R_SJe6YQWAZe6nrwt
Qualtrics ID after replacement: R_20U7XQdLf4Z3W3c
Qualtrics ID before replacement: R_BrzpINd29KjIhwt
Qualtrics ID after replacement: R_2cvVJkOPcQBHHv0


In [ ]:
test_df[test_df.index.duplicated(keep = False)].isna().sum(axis = 1)

Series([], dtype: int64)

We now need to score the questionnaires in the dataframe. To do this, we are using the column labels we loaded from a .csv file earlier to select the relevant columns.

In [ ]:
test_df.loc[:,col_key.loc['start','DASS']:col_key.loc['end','DASS']] = test_df.loc[:,col_key.loc['start','DASS']:col_key.loc['end','DASS']].replace({'Did not apply to me at all':0,'Applied to me to some degree or some of the time':1,
'Applied to me to a considerable degree or a good part of the time':2,'Applied to me very much or most of the time':3})


We now get the column names for the DASS questionnaire and divide these into subsets for depression, anxiety and stress.

In [ ]:
DASS_items = test_df.loc[:,col_key.loc['start','DASS']:col_key.loc['end','DASS']].columns

In [ ]:
DASS_depression =DASS_items[[2,4,9,12,15,16,20]] 
DASS_stress = DASS_items[[0,5,7,10,11,13,17]]
DASS_anxiety = DASS_items[[1,3,6,8,14,18,19]]

Now we calculate the scores for each of the subscale. Since we used the DASS-21 here, we have to multiply each of the scores by two.

In [ ]:
test_df['dass_depression'] = test_df.filter(items = DASS_depression,axis = 1).sum(axis = 1)*2
test_df['dass_stress'] = test_df.filter(items = DASS_stress,axis = 1).sum(axis = 1)*2
test_df['dass_anxiety'] = test_df.filter(items = DASS_anxiety,axis = 1).sum(axis = 1)*2

In [ ]:
def categorize_score(df,col_name,new_name, cutoff_low, cutoff_hi,cat_names):
    df[new_name] = df[col_name]
    for i in range(len(cutoff_low)):
        df[new_name][(df[col_name]>=cutoff_low[i])&(df[col_name]<=cutoff_hi[i])]=cat_names[i]
    return df

In [ ]:
DASS_colow = [0,10,14,21,28]
DASS_cohi = [9,13,20,27,999]
catnames = ['Normal','Mild','Moderate','Severe','Extremely Severe']
DASSa_colow = [0,8,10,15,20]
DASSa_cohi = [7,9,14,19,999]
DASSs_colow = [0,15,19,26,34]
DASSs_cohi = [14,18,25,33,999]

test_df = categorize_score(test_df,'dass_depression','dass_category_depression',DASS_colow,DASS_cohi,catnames)
test_df = categorize_score(test_df,'dass_anxiety','dass_category_anxiety',DASSa_colow,DASSa_cohi,catnames)
test_df = categorize_score(test_df,'dass_stress','dass_category_stress',DASSs_colow,DASSs_cohi,catnames)


We now go through similar steps for each of the questionnaires. The exact steps always depend on the survey in question and what we need from it.
Below, we use a different method of dividing each questionnaire into its subscales. Here, I just specified the column names. The approach depends a bit on the questionnaire (eg how long are the questions, are there special characters etc).

In [ ]:
# ERQ
# This uses sliders, so we don't need to recode this.
ERQ_reapp = ["When I want to feel more positive emotion (such as joy or amusement), I change what I'm thinking about.",
"When I want to feel less negative emotion (such as sadness or anger), I change what I'm thinking about.",
"When I'm faced with a stressful situation, I make myself think about it in a way that helps me stay calm.",
"When I want to feel more positive emotion, I change the way I'm thinking about the situation.",
"I control my emotions by changing the way I think about the situation I'm in.",
"When I want to feel less negative emotion, I change the way I'm thinking about the situation."]
ERQ_sup = ["I control my emotions by not expressing them.",
"When I am feeling negative emotions, I make sure not to express them."
"When I am feeling positive emotions, I am careful not to express them."
"I keep my emotions to myself."]

In [ ]:
test_df['ERQ_total'] = test_df.filter(items = ERQ_reapp+ERQ_sup,axis = 1).sum(axis = 1)
test_df['ERQ_reapp'] = test_df.filter(items = ERQ_reapp,axis = 1).sum(axis = 1)
test_df['ERQ_sup'] = test_df.filter(items = ERQ_sup,axis = 1).sum(axis = 1)

In [ ]:
#PTGI
test_df.loc[:,col_key.loc['start','PTGI']:col_key.loc['end','PTGI']] = test_df.loc[:,col_key.loc['start','PTGI']:col_key.loc['end','PTGI']].replace({'I did not experience this.':0,
'I experienced this to a very small degree.':1,
'I experienced this to a small degree.':2,
'I experienced this to a moderate degree.':3,
'I experienced this to a great degree.':4,
'I experienced this to a very great degree.':5})

In [ ]:
#PTGI subscales
PTGI_cols = test_df.loc[:,col_key.loc['start','PTGI']:col_key.loc['end','PTGI']].columns
test_df['PTGI_Relating to Others'] = test_df.filter(items = PTGI_cols[[5,7,8,14,15,19,20]],axis = 1).sum(axis = 1)
test_df['PTGI_New Possibilities'] = test_df.filter(items = PTGI_cols[[2,6,10,14,16]],axis = 1).sum(axis = 1)
test_df['PTGI_Personal Strength'] = test_df.filter(items = PTGI_cols[[3,9,11,18]],axis = 1).sum(axis = 1)
test_df['PTGI_Spiritual Enhancement'] = test_df.filter(items = PTGI_cols[[4,17]],axis = 1).sum(axis = 1)
test_df['PTGI_Appreciation'] = test_df.filter(items = PTGI_cols[[0,1,12]],axis = 1).sum(axis = 1)
test_df['PTGI_total'] = test_df.loc[:,col_key.loc['start','PTGI']:col_key.loc['end','PTGI']].sum(axis = 1)


In [ ]:
test_df.loc[:,col_key.loc['start','RRS']:col_key.loc['end','RRS']] = test_df.loc[:,col_key.loc['start','RRS']:col_key.loc['end','RRS']].replace({'Almost never':1,'Sometimes':2,
'Often':3,'Almost always':4})

In [ ]:
test_df['RRS_total'] = test_df.loc[:,col_key.loc['start','RRS']:col_key.loc['end','RRS']].sum(axis = 1)

For the Global Physical Activity Questionnaire, we are actually not going to use the WHO scoring system, since we are just involved in getting a measure of how much time they spend sitting/reclining or engaged in different intensities of physical activity. For each sub-category, we first convert hours to minutes, then add hours and minutes and multiply by days to get the total time per week.
We use a slightly different approach here, making an interim df for the GPAQ data. This is just what seemed easiest to me in the moment.

In [ ]:
# convert to minutes first - two ways of selecting data for variety
test_df.loc[:,test_df.columns.str.contains('Hours')]= test_df.filter(like = 'Hours',axis = 1)*60

In [ ]:
GPAQ_head = ['work_vigorous','days_work_vigorous','hours_work_vigorous','minutes_work_vigorous','work_moderate','days_work_moderate','hours_work_moderate','minutes_work_moderate',
'transport','transport_days','transport_hours','transport_minutes',
'sports_vigorous','days_sports_vigorous','hours_sports_vigorous','minutes_sports_vigorous','sports_moderate','days_sports_moderate','hours_sports_moderate','minutes_sports_moderate',
'sitting_reclining_hours','sitting_reclining_minutes']


In [ ]:
GPAQ_df = test_df.loc[:,col_key.loc['start','GPAQ']:col_key.loc['end','GPAQ']]
GPAQ_df.columns = GPAQ_head 

In [ ]:
test_df['GPAQ_vigorous_total'] = (GPAQ_df.days_work_vigorous*(GPAQ_df.hours_work_vigorous+GPAQ_df.minutes_work_vigorous))+(GPAQ_df.days_sports_vigorous*(GPAQ_df.hours_sports_vigorous+GPAQ_df.minutes_sports_vigorous))
test_df['GPAQ_moderate_total'] = (GPAQ_df.days_work_moderate*(GPAQ_df.hours_work_moderate+GPAQ_df.minutes_work_moderate))+(GPAQ_df.days_sports_moderate*(GPAQ_df.hours_sports_moderate+GPAQ_df.minutes_sports_moderate))
test_df['GPAQ_transport_total'] = GPAQ_df.transport_days*(GPAQ_df.transport_hours+GPAQ_df.transport_minutes)
test_df['GPAQ_sitting_total'] = GPAQ_df.sitting_reclining_hours+GPAQ_df.sitting_reclining_minutes

In [ ]:
#SOC - this is a rating scale so we don't need to do any conversion with a key here.
test_df['SOC'] = test_df.loc[:,col_key.loc['start','SOC']:col_key.loc['end','SOC']].sum(axis = 1)

For the PSQI, we only had two questions and only one of them needs scoring.

In [ ]:
#PSQI
test_df.loc[:,col_key.loc['end','PSQI']] = test_df.loc[:,col_key.loc['end','PSQI']].replace({'Very good':0,'Fairly good': 1, 'Fairly bad':2, 'Very bad': 3})

Now we have to deal with the ADNM scores. This will be different for the baseline and follow up files, since the ADNM was part of the follow up assessments, but not of the baseline since it was done at the screening stage. Let's first deal with the follow-up files.

In [ ]:
ADNM_preoccupation = ['I have to think about the stressful situation repeatedly.',
'I have to think about the stressful situation a lot and this is a great burden to me.',
"I constantly get memories of the stressful situation and can't do anything to stop them.",
'My thoughts often revolve around anything related to the stressful situation.']

ADNM_failure = ['Since the stressful situation, I find it difficult to concentrate on certain things.',
'Since the stressful situation, I do not like going to work or carrying out the necessary tasks in everyday life.',
'Since the situation, I can no longer sleep properly.',
'All in all, the situation causes serious impairment in my social or occupational life, my leisure time and other important areas of functioning.']

ADNM_depression = ['Since the stressful situation, I feel low and sad.',
'I rarely do those activities which I used to enjoy anymore.',
'I have been feeling dispirited since the stressful situation and have little hope for the future.']

ADNM_anxiety = ['If I think about the stressful situation, I find myself in a real state of anxiety.',
'Since the stressful situation, I am scared of doing certain things or of getting into certain situations.']

ADNM_impulse = ['I am nervous and restless since the stressful situation.',
'Since the stressful situation, I lose my temper much quicker than I used to, even over small things.',
'I have noticed that I am becoming more irritable due to the stressful situation.']

ADNM_avoidance = ['I try to avoid talking about the stressful situation whenever possible.',
'I avoid certain things that might remind me of the stressful situation.',
'I try to dismiss the stressful situation from my memory.',
'I try to suppress my feelings because they are a burden to me.']

if 'follow' in filename:
    ADNM_cols = test_df.loc[:,'Since the stressful situation, I feel low and sad.':'All in all, the situation causes serious impairment in my social or occupational life, my leisure time and other important areas of functioning.'].columns
    ADNM_items = ADNM_cols[~ADNM_cols.str.contains('How long')]



In [ ]:
if 'follow' in filename:
  test_df.loc[:,ADNM_cols] = test_df.loc[:,ADNM_cols].replace({'Never':1, 'Rarely':2,'Sometimes':3,'Often':4,'<1 month':1,'1-3 months':2, '3-6 months':3, 'more than 6 months':4})
  test_df['ADNM_time_total'] = test_df.filter(like = 'How long have you had this reaction for?', axis = 1).sum(axis = 1)
  test_df['ADNM_burden_total'] = test_df.filter(items = ADNM_items, axis = 1).sum(axis = 1)
  test_df['ADNM_preoccupation'] = test_df.filter(items = ADNM_preoccupation,axis = 1).sum(axis = 1)
  test_df['ADNM_failure'] = test_df.filter(items = ADNM_failure,axis = 1).sum(axis = 1)
  test_df['ADNM_anxiety'] = test_df.filter(items = ADNM_anxiety,axis = 1).sum(axis = 1)
  test_df['ADNM_depression'] = test_df.filter(items = ADNM_depression,axis = 1).sum(axis = 1)
  test_df['ADNM_impulse'] = test_df.filter(items = ADNM_impulse,axis = 1).sum(axis = 1)
  test_df['ADNM_avoidance'] = test_df.filter(items = ADNM_avoidance,axis = 1).sum(axis = 1)


Now for the baseline files. As mentioned, the baseline ADNM was completed at screening, so this was part of a separate survey. At this stage, participants were identified based on their screening id, which was the last four digits of their phone number. We can link them to the participant id via the randomisation spreadsheet. The reason we need to do this is that we are not interested in having all the ADNM results from the screening survey, just the ones for the participants who were actually included.
Below we combine the information from the screening and randomisation sheets to get the ADNM scores we need. Please see the comments below for further info on what each of the lines is supposed to do.

In [ ]:
# extract numeric content from strings (used to extract participant numbers)- we'll need this function below.
def find_number(text):
    num = re.findall(r'[0-9]+',text)
    return " ".join(num)

In [ ]:
if 'base' in filename:
    ADNM_cols = screening_df.loc[:,'Since the stressful situation, I feel low and sad.':'All in all, the situation causes serious impairment in my social or occupational life, my leisure time and other important areas of functioning.'].columns
    ADNM_items = ADNM_cols[~ADNM_cols.str.contains('How long')]
    randomisation_df.loc[:,['Screening Id','Participant number']] = randomisation_df.loc[:,['Screening Id','Participant number']].applymap(find_number).astype('float') # get numbers from strings and convert to float
    screening_df.rename(columns = {'Please enter the last four digits of your phone number':'Screening Id'}, inplace = True) # rename screening id column in screening file for easier manipulation
    screening_df = screening_df.replace({'Never':1, 'Rarely':2,'Sometimes':3,'Often':4,'<1 month':1,'1-3 months':2, '3-6 months':3, 'more than 6 months':4})
    screening_df['ADNM_burden_total'] = screening_df.filter(items = ADNM_items, axis = 1).sum(axis = 1)
    screening_df.drop(labels = screening_df[screening_df.ADNM_burden_total<40].index,axis = 0,inplace = True) # Some people needed several gos. Remove those duplicateswhere the total ADNM score was below the cutoff.
    duplicated_ids = screening_df[screening_df['Screening Id'].duplicated(keep = False)] # get remaining duplicated ids
    screening_df.drop(labels = duplicated_ids[duplicated_ids.ADNM_burden_total==duplicated_ids.ADNM_burden_total.min()].index,axis = 0,inplace = True) # I know there's only one more subject with duplicate values, so this will work
    screening_df.set_index('Screening Id',inplace = True) #Set index of screening file to screening id for easier manipulation
    missed_ids = randomisation_df[~randomisation_df['Screening Id'].isin(screening_df.index)]['Screening Id'] #check if any ids missing and print them
    print('the following ids are missing:', missed_ids)
    screening_df = screening_df.loc[randomisation_df['Screening Id'],:] #get data for relevant ids from screening file
    screening_df['id'] = randomisation_df.set_index('Screening Id').loc[screening_df.index,'Participant number']   #add participant number to screening df
    screening_df['ADNM_time_total'] = screening_df.filter(like = 'How long have you had this reaction for?', axis = 1).sum(axis = 1)
    screening_df['ADNM_preoccupation'] = screening_df.filter(items = ADNM_preoccupation,axis = 1).sum(axis = 1)
    screening_df['ADNM_failure'] = screening_df.filter(items = ADNM_failure,axis = 1).sum(axis = 1)
    screening_df['ADNM_anxiety'] = screening_df.filter(items = ADNM_anxiety,axis = 1).sum(axis = 1)
    screening_df['ADNM_depression'] = screening_df.filter(items = ADNM_depression,axis = 1).sum(axis = 1)
    screening_df['ADNM_impulse'] = screening_df.filter(items = ADNM_impulse,axis = 1).sum(axis = 1)
    test_df['ADNM_time_total'] = screening_df.set_index('id').loc[test_df.index,'ADNM_time_total']
    test_df['ADNM_burden_total']=screening_df.set_index('id').loc[test_df.index,'ADNM_burden_total']
    test_df['ADNM_preoccupation'] = test_df.filter(items = ADNM_preoccupation,axis = 1).sum(axis = 1)
    test_df['ADNM_failure'] = test_df.filter(items = ADNM_failure,axis = 1).sum(axis = 1)
    test_df['ADNM_anxiety'] = test_df.filter(items = ADNM_anxiety,axis = 1).sum(axis = 1)
    test_df['ADNM_depression'] = test_df.filter(items = ADNM_depression,axis = 1).sum(axis = 1)
    test_df['ADNM_impulse'] = test_df.filter(items = ADNM_impulse,axis = 1).sum(axis = 1)
    test_df['ADNM_avoidance'] = test_df.filter(items = ADNM_avoidance,axis = 1).sum(axis = 1)
    test_df = pd.concat([test_df, screening_df.set_index('id').loc[test_df.index,ADNM_event_cols]],axis = 1)
    



That's it! We now have all the info we needed from this. We are now ready to save to file.

In [ ]:
test_df.to_csv(os.path.join(output_dir,outfilename))